In [1]:
import nlopt
import numpy as np
from pybullet_suite import *

pybullet build time: Jan 27 2023 01:13:38


In [2]:
world = BulletWorld(gui=False)
sm = BulletSceneMaker(world)

argv[0]=


In [3]:
robot = world.load_robot(Panda, "panda")

In [112]:
def myfunc(x, grad, q0):
    if grad.size > 0:
        grad[:] = x - q0
    return 1/2*np.dot(x,x) - np.dot(q0, x)

def fk_const(result, x, grad, robot, pg):
    if grad.size > 0:
        grad[:] = robot.get_jacobian(x)[:3,:]
    result[:] = robot.forward_kinematics(x).trans - pg
    

In [122]:
opt = nlopt.opt(nlopt.LD_SLSQP, 7)
opt.set_lower_bounds(robot.arm_lower_limit)
opt.set_upper_bounds(robot.arm_upper_limit)

pg = np.array([0.4, -0.1, 0.3])
opt.set_min_objective(lambda x, grad: myfunc(x, grad, robot.arm_central))
opt.add_equality_mconstraint(lambda result, x, grad: fk_const(result, x, grad, robot, pg), np.ones(3)*1e-6)

opt.set_xtol_rel(1e-4)
x = opt.optimize(robot.arm_central)
minf = opt.last_optimum_value()

print("optimum at ", x)
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

optimum at  [-0.11090274  0.01097755 -0.11097191 -2.09145651 -0.04365771  1.50413172
  0.        ]
minimum value =  -2.7626042482058404
result code =  4


In [124]:
%timeit opt.optimize(robot.arm_central)

2.82 ms ± 202 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [116]:
robot.forward_kinematics(x).trans - pg

array([ 2.22331187e-09, -5.82525375e-10,  2.10222800e-09])

In [66]:
robot.inverse_kinematics(pos=pg)

array([-0.11340889,  0.0269814 , -0.11345532, -2.07467274, -0.03372916,
        1.4812015 ,  0.        ])

In [64]:
robot.forward_kinematics(x).trans

array([ 0.0972847 , -0.00220198,  0.80924302])

In [57]:
result = np.zeros(3)
grad = np.zeros((3,7))
fk_const(result, x, grad, robot, pg)
result

array([-0.4027153 ,  0.09779802,  0.50924302])

In [28]:
opt.set_min_objective(myfunc)
opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad,2,0), 1e-8)
opt.add_inequality_constraint(lambda x,grad: myconstraint(x,grad,-1,1), 1e-8)
opt.set_xtol_rel(1e-4)

x = opt.optimize([1.234, 5.678])
minf = opt.last_optimum_value()

print("optimum at ", x[0], x[1])
print("minimum value = ", minf)
print("result code = ", opt.last_optimize_result())

optimum at  0.5077784855885961 4.762529068615901
minimum value =  2.182321944309753
result code =  4


In [29]:
%timeit opt.optimize([1.234, 5.678])

125 µs ± 573 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
